## 2do Parcial - Simulación Computacional 2019-I

## 1. 
Se tiene un sistema de línea de espera con un servidor y capacidad infinita (Modelo de Colas G/G/1) (Ver [Presentación](https://docs.google.com/presentation/d/17sB_GJVIu0vJYJXovaJNdLuBMENwHMW2L9Sa58P89Uw) ), en el cual tanto los tiempos de llegada entre clientes, así como los tiempos de atención del servidor se comportan como  variables aleatorias generadas de distribuciones de probabilidad exponenciales, en donde en promedio llegan A clientes por minuto al sistema y el servidor atiende en promedio B clientes por minuto. La simulación inicia en el tiempo $t=0$ y se deja de permitir el ingreso de mas clientes en el sistema en el tiempo $T=100$.

Para analizar el sistema Ud. debe usar la implementación de acuerdo con el Modelo de Colas G/G/1 visto en clase (diapositivas 42 y 43 [Presentación](https://docs.google.com/presentation/d/17sB_GJVIu0vJYJXovaJNdLuBMENwHMW2L9Sa58P89Uw) ). 

Para la generación de las variables aleatorias con distribución exponencial debe usar el método de la inversión para variables aleatorias continuas usando generadores congruenciales mixtos. _Ayuda: El método de inversión para variables aleatorias continuas debe usar la función inversa $F^{-1}(p)$ de la distribución de probabilidad acumulada de la distribución exponencial en donde el valor $p$ proviene del generador congruencial mixto_.

Para la generación de las variables aleatorias de los tiempos entre las llegadas de clientes (variable aleatoria $X$) debe generar variables aleatorias continuas de una distribución exponencial con $\lambda = A$ a partir de un generador congruencial Borland C/C++ (i.e. $a = 22695477$, $c = 1$, $m=2^{32}$) y semilla $X_0 = 987654321$.

Para la generación de las variables aleatorias de los tiempos de atención del servidor (variable aleatoria $Y$) debe generar variables aleatorias continuas de una distribución exponencial con $\lambda = B$ a partir de un generador congruencial RANDU (i.e. $a = 65539$, $c = 0$, $m=2^{31}$) y semilla $X_0 = 123456789$.

Con su implementación para $A=3$ y $B=7$ por favor responda lo siguiente:

1. Tiempo promedio de los clientes en el sistema.
2. Tiempo promedio de los clientes en la cola.
3. Tiempo transcurrido desde $T$ hasta que el último cliente abandona el sistema.
4. Número máximo de clientes en el sistema durante la simulación.




In [16]:

import math as mt

s_entrada = 987654321
m_entrada = 2**32

s_salida = 123456789
m_salida = 2**31

def random_entrada ():
    global s_entrada, m_entrada
    s_entrada = (22695477*s_entrada + 1) % m_entrada
    return s_entrada / m_entrada

def random_salida ():
    global s_salida, m_salida
    s_salida = (65539*s_salida) % m_salida
    return s_salida / m_salida

def arrives(rate):
    return -mt.log(1-random_entrada())/rate

def serves(rate):
    return -mt.log(1-random_salida())/rate

T = int(1e2)

t = t_suc = N_LL = N_S = n = 0
T_LL = T_S = mt.inf

LL = []
S = []
Serv = []
VA = []

X = 0

A = 7
B = 3

def Llegada(t_suc):
    global t, n, N_LL, LL, Serv, T_S, T_LL, VA, A, B
    
    t = t_suc
    n = n + 1
    N_LL = N_LL + 1
    LL.append(t)
    X = arrives(A)
    
    if ((t + X) < T):
        T_LL = t + X
    if(n == 1):
        Y = serves(B)
        T_S = t + Y
        Serv.append(Y)

def Servidor(t_suc):
    global t, n, S, N_S, Serv, T_S, A, B
    
    t = t_suc
    n = n - 1
    N_S = N_S + 1
    S.append(t)
    
    if(n > 0):
        Y = serves(B)
        T_S = t + Y
        Serv.append(Y)

        
        
X = arrives(A)

if(X > T):
    T_p = t_med_sistema = t_med_cola = 0
    sys.exit(0)
else:
    Llegada(X)
    while(T_LL != mt.inf or T_S != mt.inf):
        if(T_LL < T_S):
            t_suc = T_LL
            T_LL = mt.inf
            Llegada(t_suc)
        if(T_S < T_LL):
            t_suc = T_S
            T_S = mt.inf
            Servidor(t_suc)
            
T_p = max(0, t-T)
cum_1 = cum_2 = 0
ind = 0
    
for ind in range(N_LL):
    cum_1 = cum_1 + S[ind] - LL[ind]
    cum_2 = cum_2 + S[ind] - LL[ind] - Serv[ind]
    ind = ind + 1
    
t_med_sistema = cum_1/N_LL
t_med_cola = cum_2/N_LL

print("Tiempo promedio de los clientes en el sistema = ", t_med_sistema)
print("Tiempo promedio de los clientes en la cola", t_med_cola)
print(T_p)
print("# Clientes atendidos:", N_S)






Tiempo promedio de los clientes en el sistema =  72.62544708127432
Tiempo promedio de los clientes en la cola 72.29961646597414
136.63840189192564
# Clientes atendidos: 726


## 2.
Usando el ejemplo del modelo de simulación basada en agentes del comportamiento de hormigas para buscar comida y llevarla al nido de NetLogo en su versión online "Sample Models/Biology/Ants" (http://www.netlogoweb.org/launch#http://www.netlogoweb.org/assets/modelslib/Sample%20Models/Biology/Ants.nlogo), modifique el código de tal forma que:


In [ ]:
patches-own [
  chemical             ;; amount of chemical on this patch
  food                 ;; amount of food on this patch (0, 1, or 2)
  nest?                ;; true on nest patches, false elsewhere
  nest-scent           ;; number that is higher closer to the nest
  food-source-number   ;; number (1, 2, or 3) to identify the food sources
]

;;;;;;;;;;;;;;;;;;;;;;;;
;;; Setup procedures ;;;
;;;;;;;;;;;;;;;;;;;;;;;;

to setup
  clear-all
  set-default-shape turtles "bug"
  create-turtles population
  [ set size 2         ;; easier to see
    set color red  ]   ;; red = not carrying food
  setup-patches
  reset-ticks
end

to setup-patches
  ask patches
  [ setup-nest
    setup-food
    recolor-patch ]
end

to setup-nest  ;; patch procedure
  ;; set nest? variable to true inside the nest, false elsewhere
  set nest? (distancexy 0 0) < 5
  ;; spread a nest-scent over the whole world -- stronger near the nest
  set nest-scent 200 - distancexy 0 0
end

to setup-food  ;; patch procedure
  ;; setup food source one on the right
  if (distancexy (0.6 * max-pxcor) 0) < 5
  [ set food-source-number 1 ]
  ;; setup food source two on the lower-left
  if (distancexy (-0.6 * max-pxcor) (-0.6 * max-pycor)) < 5
  [ set food-source-number 2 ]
  ;; setup food source three on the upper-left
  if (distancexy (-0.8 * max-pxcor) (0.8 * max-pycor)) < 5
  [ set food-source-number 3 ]
  ;; set "food" at sources to either 1 or 2, randomly
  if food-source-number > 0
  [ set food one-of [1 2] ]
end

to recolor-patch  ;; patch procedure
  ;; give color to nest and food sources
  ifelse nest?
  [ set pcolor violet ]
  [ ifelse food > 0
    [ if food-source-number = 1 [ set pcolor cyan ]
      if food-source-number = 2 [ set pcolor sky  ]
      if food-source-number = 3 [ set pcolor blue ] ]
    ;; scale color to show chemical concentration
    [ set pcolor scale-color green chemical 0.1 5 ] ]
end

;;;;;;;;;;;;;;;;;;;;;
;;; Go procedures ;;;
;;;;;;;;;;;;;;;;;;;;;

to go  ;; forever button
  ask turtles
  [ if who >= ticks [ stop ] ;; delay initial departure
    ifelse color = red
    [ look-for-food  ]       ;; not carrying food? look for it
    [ return-to-nest ]       ;; carrying food? take it back to nest
    wiggle
    fd 1 ]
  diffuse chemical (diffusion-rate / 100)
  ask patches
  [ set chemical chemical * (100 - evaporation-rate) / 100  ;; slowly evaporate chemical
    recolor-patch ]
  tick
end

to return-to-nest  ;; turtle procedure
  ifelse nest?
  [ ;; drop food and head out again
    set color red
    rt 180 ]
  [ set chemical chemical + 15  ;; drop some chemical
    uphill-nest-scent ]         ;; head toward the greatest value of nest-scent
end

to look-for-food  ;; turtle procedure
  if food > 0
  [ set color orange + 1     ;; pick up food
    set food food - 1        ;; and reduce the food source
    rt 180                   ;; and turn around
    stop ]
  ;; go in the direction where the chemical smell is strongest
  if (chemical >= 0.025) and (chemical < 4)
  [ uphill-chemical ]
end

;; sniff left and right, and go where the strongest smell is
to uphill-chemical  ;; turtle procedure
  let scent-ahead chemical-scent-at-angle   0
  let scent-right chemical-scent-at-angle  45
  let scent-left  chemical-scent-at-angle -45
  if (scent-right > scent-ahead) or (scent-left > scent-ahead)
  [ ifelse scent-right > scent-left
    [ rt 45 ]
    [ lt 45 ] ]
end

;; sniff left and right, and go where the strongest smell is
to uphill-nest-scent  ;; turtle procedure
  let scent-ahead nest-scent-at-angle   0
  let scent-right nest-scent-at-angle  45
  let scent-left  nest-scent-at-angle -45
  if (scent-right > scent-ahead) or (scent-left > scent-ahead)
  [ ifelse scent-right > scent-left
    [ rt 45 ]
    [ lt 45 ] ]
end

to wiggle  ;; turtle procedure
  rt random 40
  lt random 40
  if not can-move? 1 [ rt 180 ]
end

to-report nest-scent-at-angle [angle]
  let p patch-right-and-ahead angle 1
  if p = nobody [ report 0 ]
  report [nest-scent] of p
end

to-report chemical-scent-at-angle [angle]
  let p patch-right-and-ahead angle 1
  if p = nobody [ report 0 ]
  report [chemical] of p
end


; Copyright 1997 Uri Wilensky.
; See Info tab for full copyright and license.



1. La feromona dejada por las hormigas al encontrar comida en su rastro sea de 15.

In [ ]:
[ set chemical chemical + 15  ;; drop some chemical

2. El intervalo de la cantidad de la feromona para que una hormiga siga la dirección del rastro sea entre 0.025 y 4.

In [ ]:
if (chemical >= 0.025) and (chemical < 4)


3. ¿Qué cambio en relación al modelo original respecto al comportamiento de las hormigas y su objetivo de encontrar comida y llevarla al nido?

las hormigas demoraron 1200ticks aprox consumir las 3 fuentes de comida para llevarlas a los nidos por la buena cantidad de feromona apenas encontraban una fuente la terminaban con los siguientes tiempos:

* PRIMERA FUENTE = 200 ticks aprox
* SEGUNDA FUENTE = 700 ticks aprox 
* TERCERA FUENTE = 1200 ticks aprox

<img src="antorig.png" height="200" width="200">

Para el cambio de parametros, la feromona es mas baja por lo que el rastro no dura mucho, cosa que hace que se borren los "caminos". aqui duro mucho mas en terminar las fuentes, obteniendo los sgtes resultados:

* PRIMERA FUENTE = 260 ticks aprox
* SEGUNDA FUENTE = 2200 ticks aprox 
* TERCERA FUENTE = 4000 ticks aprox

<img src="antmod.png" height="200" width="200">

## 3.
Tomando en cuenta la ecuación para actualizar la feromona en  (Ver Diap. 17 [Presentación](http://mat.uab.es/~alseda/MasterOpt/ACO_Intro.pdf) ), en el 

\begin{align*}
\tau _{i,j}= (1-\rho )\tau _{i,j}+\sum_{k=1}^{m}\Delta \tau_{i,j}^{k}
\end{align*}

donde 

$\tau _{i,j}$ es la cantidad de feromona en un arco (vertice) dado entre los nodos $i,j$

$\rho$ es la tasa de evaporación de la feromona

$m$ es el número de hormigas

$\Delta \tau_{i,j}^{k}$ es la cantidad de feromona depositada en el arco ($i,j$) por la $k$-ésima hormiga 

\begin{align*}
\Delta \tau_{i,j}^{k}=
\begin{cases}
1/L_k & \text{ si la hormiga }k\text{ viaja en el arco } i,j \\ 
0 & \text{ si no }
\end{cases}
\end{align*}

donde $L_k$ es la longitud (distancia) de la trayectoria de la $k$-ésima hormiga 

Implemente una función para la actualización de la feromona que reciba por parametros los valores $\tau _{i,j}$ y $\rho$, y el un arreglo de tamaño $m$ para cada uno de los $\Delta \tau_{i,j}^{k}$


In [19]:
Txy=0
Lk=0
def sum(Txy):
    if():
        return 1/Lk
    else:
        return 0


def update_pheromone (p) :
    global Txy
    Txy = [(1-p)*Txy] + sum(Txy)
    return 0


## 4.

Escriba el código en NetLogo que permita recibir por parámetro la longitud de lado para dibujar un hexagono, y  presente la captura de pantalla del resultado obtenido.

In [ ]:
to hexagono[lado]
  ca
  crt 1
  let n 6
  let ang 360 / n
  ask turtles [repeat n [pd fd lado rt ang]]
end

Screen

<img src="netl.png" height="200" width="200">

## 5.
Aplicando las reglas del juego de la vida de Conway para el siguiente entorno, ¿Cuál sería el estado del sistema luego de 5 iteraciones?
<img src="9x9grid.jpg" height="200" width="200">

En la 5ta iteracion

<img src="lifegame.png" height="200" width="200">
